In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Wed May 18 08:40:19 2022

@author: MREID7
"""

import os

#Maps Users to Market or Market Group
STR_SQL_USERS_TO_MKT_MKTGRP = '''select distinct coalesce(m.MarketName,mg.MarketGroupName) as MktOrMktGrp, u.PIN, butmomg.MarketOrMarketGroupUUID,u.UserUUID 
FROM User u
INNER JOIN Employment e on u.UserUUID  = e.UserUUID 
INNER JOIN BusinessUnitToMarketOrMarketGroup butmomg on e.BusinessUnitUUID = butmomg.BusinessUnitUUID 
LEFT JOIN Market m on butmomg.MarketOrMarketGroupUUID = m.MarketUUID 
LEFT JOIN MarketGroup mg on butmomg.MarketOrMarketGroupUUID = mg.MarketGroupUUID 
WHERE u.Active = b'1'
AND e.EmploymentStatusUUID = (SELECT es.EmploymentStatusUUID  from EmploymentStatus es WHERE es.Name = 'Employed')
'''

#Fast track users
STR_SQL_FASTTRACK_USERS = '''SELECT DISTINCT 
	  u.PIN
	, jrl.JobRoleName
	, GROUP_CONCAT(DISTINCT COALESCE(mg.MarketGroupName, m.MarketName)) AS Markets
	, GROUP_CONCAT(DISTINCT ljl2.Title) AS LearnerJourneys
	, GROUP_CONCAT(DISTINCT ljll.LearnerJourneyLevelTitle) AS FastTrackLevels
	, GROUP_CONCAT(DISTINCT bl.BrandName) AS Brands
FROM UserCertificationPathFastTrackLevel ucpftl
INNER JOIN UserCertificationPath ucp USING (UserCertificationPathUUID)
INNER JOIN EmploymentJobRole ejr USING (EmploymentJobRoleUUID)
INNER JOIN Employment e USING (EmploymentUUID)
INNER JOIN BusinessUnit bu ON e.BusinessUnitUUID = bu.BusinessUnitUUID 
INNER JOIN BrandLocal bl ON bu.BrandUUID = bl.BrandUUID
INNER JOIN User u ON e.UserUUID = u.UserUUID
INNER JOIN BusinessUnitToMarketOrMarketGroup butmomg ON bu.BusinessUnitUUID = butmomg.BusinessUnitUUID
INNER JOIN EmploymentJobRoleStatus ejrs USING (EmploymentJobRoleStatusUUID)
INNER JOIN EmploymentStatus es USING (EmploymentStatusUUID)
INNER JOIN UserStatus us ON u.UserAccountStatusUUID = us.UserStatusUUID
INNER JOIN JobRoleLocal jrl ON ejr.JobRoleUUID = jrl.JobRoleUUID AND jrl.LanguageUUID = (SELECT LanguageUUID FROM Language WHERE LanguageName = 'English')
INNER JOIN LearnerJourneyLevel ljl ON ucpftl.LearnerJourneyLevelUUID = ljl.LearnerJourneyLevelUUID AND ljl.IsFastTrack = 1
INNER JOIN LearnerJourneyLevelLocal ljll ON ljl.LearnerJourneyLevelUUID = ljll.LearnerJourneyLevelUUID AND ljll.LanguageUUID = (SELECT LanguageUUID FROM Language WHERE LanguageName = 'English')
INNER JOIN LearnerJourneyVersion ljv ON ljl.LearnerJourneyVersionUUID = ljv.LearnerJourneyVersionUUID AND ucp.LearnerJourneyUUID = ljv.LearnerJourneyUUID
INNER JOIN LearnerJourneyLocal ljl2 ON ljv.LearnerJourneyVersionUUID = ljl2.LearnerJourneyVersionUUID AND ljl2.LanguageUUID = (SELECT LanguageUUID FROM Language WHERE LanguageName = 'English')
LEFT JOIN MarketGroup mg ON butmomg.MarketOrMarketGroupUUID = mg.MarketGroupUUID
LEFT JOIN Market m ON butmomg.MarketOrMarketGroupUUID = m.MarketUUID
WHERE ejrs.Name = 'Active' AND es.Name = 'Employed' AND us.Name = 'Active' AND ucpftl.IsFastTrack = 1
GROUP BY u.PIN, jrl.JobRoleName '''

#Learner Journey Level History
STR_SQL_JOURNEY_LEVEL_HISTORY = '''select u.PIN, cp.Name as CertPath, jr.ReferenceCode as JobRole, ujlh.StartDate, ujlh.CompletionDate, ujlh.LevelExpiryDate,ujlh.CreatedDate,ljll.LearnerJourneyLevelTitle , cs.StatusCode, ujlh.IsCurrent,cp.IsTechnical, ujlh.UserJourneyLevelHistoryUUID 
from UserJourneyLevelHistory ujlh
inner join User u on ujlh.UserUUID = u.UserUUID 
inner join CourseStatus cs on ujlh.SuccessStatusUUID = cs.CourseStatusUUID 
inner join LearnerJourneyLevelLocal ljll on ujlh.LearnerJourneyLevelUUID = ljll.LearnerJourneyLevelUUID  and ljll.LanguageUUID = '33e5850a-473f-11e8-9661-7bf55cea83d1'
inner join LearnerJourneyVersion ljv on ujlh.LearnerJourneyVersionUUID = ljv.LearnerJourneyVersionUUID 
inner join JobRole jr on ljv.JobRoleUUID = jr.JobRoleUUID 
inner join CertificationPath cp on ljv.CertificationPathUUID = cp.CertificationPathUUID 
left join EmploymentJobRole ejr on ujlh.UserEntityJobRoleUUID = ejr.EmploymentJobRoleUUID 
where cp.IsTechnical > 0'''

#FROM EVO_FIMS_Combo_02
STR_SQL_BUSINESS_UNITS = '''SELECT COALESCE(m.MarketName, mg.MarketGroupName) AS MarketName, but.Name as BusUnitType, bust.Name as BusUnitSubType, bus.Name as UnitStatus, CONV(bus.IsActiveStatus,2,10) AS IsActiveStatus, bu.Name as BusinessUnit, b.BrandCode,  bu.CICode, a.Town, bu.ChargingStartDate, bu.ChargingEndDate, bu.BusinessUnitUUID
FROM BusinessUnit bu
INNER JOIN BusinessUnitType but ON bu.BusinessUnitTypeUUID = but.BusinessUnitTypeUUID 
INNER JOIN BusinessUnitSubType bust ON bu.BusinessUnitSubTypeUUID = bust.BusinessUnitSubTypeUUID 
INNER JOIN BusinessUnitStatus bus ON bu.BusinessUnitStatusUUID = bus.BusinessUnitStatusUUID 
LEFT JOIN Address a ON bu.AddressUUID = a.AddressUUID 
LEFT JOIN Brand b ON bu.BrandUUID = b.BrandUUID 
LEFT JOIN BusinessUnitToMarketOrMarketGroup butmomg on bu.BusinessUnitUUID = butmomg.BusinessUnitUUID 
LEFT JOIN Market m on butmomg.MarketOrMarketGroupUUID = m.MarketUUID
LEFT JOIN MarketGroup mg on butmomg.MarketOrMarketGroupUUID = mg.MarketGroupUUID'''

#FROM EVO_Reporting_Extracts_05
STR_SQL_USERS = '''SELECT u.PIN, up.FirstName, up.Surname, up.EmailAddress, CONV(u.Active,2,10) as Active, u.LastLogin, uss.Name as UserSubStatus, u.UserUUID
FROM JLR_LMS_Reporting.User u
INNER JOIN UserProfile up ON u.UserUUID = up.UserUUID
INNER JOIN UserSubStatus uss ON u.UserAccountSubStatusUUID = uss.UserSubStatusUUID'''

STR_SQL_EMPLOYMENT = '''SELECT ES.Name as EmpStatus, b.BrandCode, CONV(E.IsMerged,2,10) AS IsMerged, E.BusinessUnitUUID, E.UserUUID, E.EmploymentUUID, E.StartDate, E.EndDate, E.CreatedDate, E.UpdatedDate  
FROM JLR_LMS_Reporting.Employment E
INNER JOIN EmploymentStatus ES ON ES.EmploymentStatusUUID = E.EmploymentStatusUUID
LEFT JOIN Brand b ON E.BrandUUID = b.BrandUUID '''


STR_SQL_EMPLOYMENT_JOB_ROLE = '''select jr.ReferenceCode, e.EmploymentUUID, es.Name as EmpStatus, CONV(ejr.IsPrimary,2,10) AS IsPrimary, ejr.StartDate, ejr.EndDate, b.BrandCode, CONV(e.IsMerged,2,10) AS IsMerged, jr.CertificationPath, jr.CertificationPathUUID, e.BusinessUnitUUID, e.UserUUID, ejr.ManagerUUID, jr.DepartmentUUID, ejr.EmploymentJobRoleUUID, ejrs.Name EJRstatus 
FROM EmploymentJobRole ejr 
INNER JOIN Employment e ON ejr.EmploymentUUID  = e.EmploymentUUID
INNER JOIN EmploymentStatus es ON e.EmploymentStatusUUID = es.EmploymentStatusUUID
INNER JOIN EmploymentJobRoleStatus ejrs ON ejr.EmploymentJobRoleStatusUUID = ejrs.EmploymentJobRoleStatusUUID 
INNER JOIN JobRole jr ON ejr.JobRoleUUID = jr.JobRoleUUID
LEFT JOIN Brand b ON e.BrandUUID = b.BrandUUID'''

# STR_SQL_EMPLOYMENT_JOB_ROLE = '''select jr.ReferenceCode, es.Name as EmpStatus, CONV(ejr.IsPrimary,2,10) AS IsPrimary, ejr.StartDate, ejr.EndDate, b.BrandCode, CONV(e.IsMerged,2,10) AS IsMerged, jr.CertificationPath, jr.CertificationPathUUID, e.BusinessUnitUUID, e.UserUUID, ejr.ManagerUUID, jr.DepartmentUUID, ejr.EmploymentJobRoleUUID, ejrs.Name EJRstatus 
# FROM EmploymentJobRole ejr 
# INNER JOIN Employment e ON ejr.EmploymentUUID  = e.EmploymentUUID
# INNER JOIN EmploymentStatus es ON e.EmploymentStatusUUID = es.EmploymentStatusUUID
# INNER JOIN EmploymentJobRoleStatus ejrs ON ejr.EmploymentJobRoleStatusUUID = ejrs.EmploymentJobRoleStatusUUID 
# INNER JOIN JobRole jr ON ejr.JobRoleUUID = jr.JobRoleUUID
# LEFT JOIN Brand b ON e.BrandUUID = b.BrandUUID'''

STR_SQL_CERIFICATION_PATH = '''select cp.Name as CertPath, cc.CertifiedLevel as CurrentLevelName, cc.TargetLevel as TargetLevelName, cc.CertificationExpiryDate as ExpirationDate, 
CONV(cp.IsTechnical,2,10) as IsTechnical,  CONV(ucp.IsFastTrack,2,10) as IsFastTrack, ucp.LearnerJourneyUUID, ucp.EmploymentJobRoleUUID,
ucp.UserUUID, ujlh.CompletionDate, u.PIN, cc.AssociateLevel as CurrentStudyLevel
from Cache_Certification cc 
inner join UserCertificationPath ucp on cc.EmploymentJobRoleUUID = UUID_TO_BIN(ucp.EmploymentJobRoleUUID,TRUE) 
inner join EmploymentJobRole ejr2 on ucp.EmploymentJobRoleUUID = ejr2.EmploymentJobRoleUUID 
inner join EmploymentJobRoleStatus ejrs on ejr2.EmploymentJobRoleStatusUUID = ejrs.EmploymentJobRoleStatusUUID 
inner join CertificationPath cp on ucp.CertificationPathUUID = cp.CertificationPathUUID 
inner join User u on ucp.UserUUID = u.UserUUID 
left join UserJourneyLevelHistory ujlh on ucp.EmploymentJobRoleUUID = ujlh.UserEntityJobRoleUUID 
and ujlh.LearnerJourneyLevelUUID = ucp.CurrentLevelUUID 
and ujlh.IsCurrent = b'1'
where ejrs.Name = 'Active'
order by u.PIN, cc.TargetLevel '''

STR_SQL_CERTIFICATION_SUMMARY = '''select cp.Name as CertPath, u.PIN, ljll.LearnerJourneyLevelTitle, cs.StatusCode  as LJstatus,  
ujlh.CompletionDate as DateAchieved, ujlh.LevelExpiryDate as ExpiryDate, CONV(cp.IsTechnical,2,10) as IsTechnical, 
CONV(ljl.IsFastTrack,2,20) as IsFastTrack,ujlh.UserUUID, ljv.LearnerJourneyVersionUUID 
from UserJourneyLevelHistory ujlh 
inner join User u on ujlh.UserUUID = u.UserUUID
inner join UserProfile up on u.UserUUID = up.UserUUID 
inner join LearnerJourneyVersion ljv on ujlh.LearnerJourneyVersionUUID = ljv.LearnerJourneyVersionUUID 
inner join CourseStatus cs on ujlh.SuccessStatusUUID = cs.CourseStatusUUID 
inner join LearnerJourneyLevel ljl on ujlh.LearnerJourneyLevelUUID = ljl.LearnerJourneyLevelUUID
inner join LearnerJourneyLevelLocal ljll ON ljl.LearnerJourneyLevelUUID = ljll.LearnerJourneyLevelUUID AND ljll.LanguageUUID = (SELECT LanguageUUID FROM Language WHERE LanguageName = 'English')
inner join CertificationPath cp on ljv.CertificationPathUUID = cp.CertificationPathUUID 
where ujlh.IsCurrent = 1
order by u.PIN, ujlh.CompletionDate'''

#FROM V2E_Views_Data_08
STR_SQL_LEARNING_CLASSROOM = '''select cu.PIN, CONV(cuc.IsCurrent,2,10) as IsCurrent, cuc.AssetTypeName, cuc.CourseCode, cuc.CourseTitle, cuc.ClassroomSessionSessionName, cuc.ClassroomSessionStartDate, cuc.ClassroomSessionEndDate, cuc.ClassroomSessionDuration, cuc.Status, cuc.CourseStatusName, cuc.Score, cuc.PreReqScore, cuc.DirScore, cuc.EndTestScore, cuc.OveralScore, cuc.VenueName, cuc.AttempDate, cuc.CompletedDate, cuc.CourseLanguageName, cuc.Version, cuc.ClassroomPreReqCount
from Cache_UserClassroom cuc INNER JOIN Cache_User cu ON cuc.UserUUID = cu.UserUUID'''


STR_SQL_LEARNING_EXAMS = '''select cu.PIN, CONV(cue.IsCurrent,2,10) as IsCurrent, cue.AssetTypeName, cue.CourseCode, cue.CourseTitle, cue.ClassroomSessionSessionName, 
cue.ClassroomSessionStartDate, cue.ClassroomSessionEndDate, cue.ClassroomSessionDuration, cue.Status, 
cue.CourseStatusName, cue.Score, cue.PreReqScore, cue.DirScore, cue.EndTestScore, cue.OveralScore, cue.VenueName, 
cue.AttempDate, cue.CompletedDate, cue.CourseLanguageName, cue.Version, cue.ClassroomPreReqCount
from Cache_UserExam cue 
INNER JOIN Cache_User cu ON cue.UserUUID = cu.UserUUID'''

STR_SQL_LEARNING_ONLINE = '''select cu.PIN, CONV(cuol.IsCurrent,2,10) as IsCurrent, cuol.AssetTypeName, cuol.CourseCode, cuol.CourseTitle, cuol.ClassroomSessionSessionName, cuol.ClassroomSessionStartDate, cuol.ClassroomSessionEndDate, cuol.ClassroomSessionDuration, 
cuol.Status, cuol.CourseStatusName, cuol.Score, cuol.PreReqScore, cuol.DirScore, cuol.EndTestScore, cuol.OveralScore, cuol.VenueName, 
cuol.AttempDate, cuol.CompletedDate, cuol.CourseLanguageName, cuol.Version, cuol.ClassroomPreReqCount
from Cache_UserOnlineLearning cuol 
INNER JOIN Cache_User cu ON cuol.UserUUID = cu.UserUUID '''

FILE_DEST_BUSINESS_UNITS = 'BusUnits.csv'
FILE_DEST_USERS = 'Users.csv'
FILE_DEST_EMPLOYMENT = 'Employment.csv'
FILE_DEST_EMPLOYMENT_JOB_ROLE = 'EmpJobRole.csv'
FILE_DEST_CERTIFICATION_PATH = 'CertPath.csv'
FILE_DEST_CERTIFICATION_SUMMARY = 'CertsSummary.csv'
FILE_DEST_LEARNING_CLASSROOM = 'ClassroomLearningHistory.csv'
FILE_DEST_LEARNING_EXAMS = 'UserExamHistory.csv'
FILE_DEST_LEARNING_ONLINE = 'OnlineLearningHistory.csv'
FILE_DEST_FASTTRACKS = 'FastTrackUsers.csv'
FILE_DEST_JOURNEY_LEVEL_HISTORY = 'TechLevelsHistory.csv'
FILE_SRC_FIMS_SPECIALISM = 'Franchise_Specialism_Report.xlsx'
FILE_SRC_FIMS_NETWORK = 'Network_Planning_Report.xlsx'


DIC_QUERIES_FILES_01 = {STR_SQL_USERS:FILE_DEST_USERS,STR_SQL_EMPLOYMENT:FILE_DEST_EMPLOYMENT,STR_SQL_EMPLOYMENT_JOB_ROLE:FILE_DEST_EMPLOYMENT_JOB_ROLE,STR_SQL_CERIFICATION_PATH:FILE_DEST_CERTIFICATION_PATH,STR_SQL_CERTIFICATION_SUMMARY:FILE_DEST_CERTIFICATION_SUMMARY}
DIC_QUERIES_FILES_02 = {STR_SQL_LEARNING_CLASSROOM:FILE_DEST_LEARNING_CLASSROOM,STR_SQL_LEARNING_EXAMS:FILE_DEST_LEARNING_EXAMS,STR_SQL_LEARNING_ONLINE:FILE_DEST_LEARNING_ONLINE}

STR_DB_IP_PROD = '99.81.195.113'
STR_DB_PORT_PROD = '13306'
STR_DB_NAME_PROD = 'JLR_LMS_Reporting'

def Username():
    return os.getlogin().lower()

def DIR_REPOSITORY():
    return 'C:\\Users\\' + Username() + '\\OneDrive\\JAGUAR LAND ROVER\\Network Excellence Reporting & Analytics - NERA Data Repository\\'

def DIR_EXTRACTS():
    return DIR_REPOSITORY() + 'Raw Extracts\\'

def DIR_RAW_EVO_EXTRACTS():
    return DIR_EXTRACTS() + 'EvoReportingDB\\'

def DIR_FIMS_SPECIALISM_EXTRACTS():
    return DIR_EXTRACTS() + 'FIMS - Franchise Specialism Report\\'

def DIR_FIMS_NETWORK_EXTRACTS():
    return DIR_EXTRACTS() + 'FIMS - Network Development Report\\'


STR_TEMP_FOLDER = 'C:\\luigi temp\\'
# Think this is not needed STR_RSA_PATH = 'C:\\Program Files\\RSA SecurID Software Token\\SecurID.exe'
STR_RAPIDMINER_PATH = 'C:\\Program Files\\RapidMiner\\RapidMiner Studio\\scripts\\rapidminer-batch.bat'
STR_NERA_Repository = r'//NERA RapidMiner Repository'
STR_RM_SUCCESS_STRING = 'finished successfully after'


DB_ENVIRONMENT =  'V2E_Reporting_PROD' #'V2E_Warehouse_UAT' #'V2E_Reporting_UAT' #'V2E_Reporting_PROD'
FILE_CREDENTIALS = 'local_configs.json'
crdntlspath = os.path.expanduser('C:\\Users\\' + Username() + '\\OneDrive\\OneDrive - JAGUAR LAND ROVER\Documents' + '\\' + FILE_CREDENTIALS)

if __name__ == '__main__':
    print('Running Module NERA_BAU_STATIC_DATA_01')

In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Wed May 18 09:11:03 2022

@author: MREID7
"""
#import os
#import re
from datetime import datetime
#import time
import json
#import pymysql
import pandas as pd
import sqlalchemy as sqa
from lxml import etree
from lxml.objectify import NoneElement
import lxml
import NERA_BAU_STATIC_DATA_01 as STATIC_DATA

print(STATIC_DATA.Username())
DATE_FORMAT = '%Y-%m-%d %H:%M:%S'


USER_DOCS_PATH = 'C:\\Users\\' + STATIC_DATA.Username() + '\\OneDrive\\OneDrive - JAGUAR LAND ROVER\\Documents'
NERA_REPOS_PATH = 'C:\\Users\\' + STATIC_DATA.Username() + r'\OneDrive\JAGUAR LAND ROVER\Network Excellence Reporting & Analytics - NERA Data Repository'
FILE_FIMS_SPEC = NERA_REPOS_PATH + '\\Raw Extracts\\FIMS - Franchise Specialism Report\Franchise_Specialism_Report.xlsx'
FIL_STATIC = NERA_REPOS_PATH + '\\Reference Data\Business Units\\FIMsStaticMapping.json'
DIR_RAPIDMINER = NERA_REPOS_PATH + '\\Raw Extracts\\EvoReportingDB'

DB_DESC =  'V2E_Reporting_UAT' #'V2E_Warehouse_UAT' #'V2E_Reporting_UAT' # 'V2E_Reporting_PROD'
FILE_CREDENTIALS = 'local_configs.json'

def GetDBengine(sKeyDB,sCredsPath,sCredsFile):
    #Creates a database connection and SQLAlchemy engine from User Config file
    crdntlspath = sCredsPath + '\\' + sCredsFile
    with open(crdntlspath,'r') as read_crdntls:
        crdntls = json.load(read_crdntls)
        dbentry = crdntls['logins'][sKeyDB]
    connStr = 'mysql+pymysql://' + dbentry['user'] + ':' + dbentry['pass'] + '@' + dbentry['ip'] + ':' + dbentry['port'] + '/' + dbentry['name']
    objEngine = sqa.create_engine(connStr)
    return objEngine
    

def GetFIMsAddressIDoverrides():
    with open(FIL_STATIC,'r') as objSTATIC:
        dicStatic = json.load(objSTATIC)
        dicADDRESSID_OVERRIDES = dicStatic['AddressIDmapping']
    return dicADDRESSID_OVERRIDES

def GetFIMsMarketsOverrides():
    with open(FIL_STATIC,'r') as objSTATIC:
        dicStatic = json.load(objSTATIC)
        dicFIMS_EVO_MKT_MAP = dicStatic['MarketMapping']
    return dicFIMS_EVO_MKT_MAP

def GetEvoBusinessUnits():
    engine = GetDBengine(DB_DESC,USER_DOCS_PATH,FILE_CREDENTIALS)
    strSQL = STATIC_DATA.STR_SQL_BUSINESS_UNITS
    dfEvo2 = pd.read_sql(strSQL,engine)
    dfEvo2['IsRetailer'] = dfEvo2['BusUnitSubType'].map(lambda x: 1 if (('Retailer' in x )|('Sales' in x)|('Independent' in x))else 0)
    dfEvo2['FranCICode'] = dfEvo2['CICode'].map(lambda x: x[-5::] if len(x) >4 else x)
    dfEvo2['BrandCode'] = dfEvo2['BrandCode'].str[0]
    return dfEvo2

def GetFIMsBusUnits():
    dfFIMSSrc = pd.read_excel(FILE_FIMS_SPEC,sheet_name='Report 1',skiprows=3)
    dfFIMSSrc.drop(dfFIMSSrc.columns.difference(['Distributor CI Code','Franchise Country','Franchise CI Code','JLR Number','Franchise Trading Title','Brand','Franchise Status','Franchise Type','Address Postcode','Address Town','Address ID']),axis=1,inplace=True)
    dicManualMarkets = GetFIMsMarketsOverrides()
    dfFIMSSrc.replace({"Franchise Country": dicManualMarkets},inplace=True)
    dfFIMSSrc['Brand'] = dfFIMSSrc['Brand'].str[0]
    dfFIMSSrc['F_CICode'] = dfFIMSSrc['Distributor CI Code'] + dfFIMSSrc['Franchise CI Code']
    dfFIMSSrc['IsRetailer'] = dfFIMSSrc['Franchise Type'].map(lambda x: 1 if (('Retailer' in x )|('Sales' in x)|('Boutique' in x))else 0)
    return dfFIMSSrc

def GetCombinedFIMsEvo():
    dfFIMS = GetFIMsBusUnits()
    dfEvo = GetEvoBusinessUnits()
    dfMerged01 = dfEvo.merge(dfFIMS, how='left', left_on=['FranCICode','BrandCode','MarketName','IsRetailer'], right_on=['Franchise CI Code','Brand','Franchise Country','IsRetailer'])
    dfMerged01['ToRemove'] = dfMerged01.apply(lambda x: 1 if ((x['IsActiveStatus'] ==0 )&(x['Franchise Status'] == 'Terminated'))else 0,axis=1)
    dicAddressIDOverrides = GetFIMsAddressIDoverrides()
    dfMerged01['AddressID']=dfMerged01.apply(lambda x: dicAddressIDOverrides[x['CICode']]  if x['CICode'] in dicAddressIDOverrides else x['Address ID'] ,axis=1)
    #As a final capture - those whose CI Codes are exactly the same but have not yet been mapped (bedcause the Franchise type is different)
    dicLastChanceMaps = {}
    for indx,row in dfFIMS[dfFIMS['Franchise Status']=='Active'][['F_CICode','Address ID','IsRetailer']].iterrows():
        if row['F_CICode'] in dicLastChanceMaps:
            if row['IsRetailer'] ==1:
                dicLastChanceMaps[row['F_CICode']]=row['Address ID']
        else:
            dicLastChanceMaps[row['F_CICode']]=row['Address ID']
    dfMerged01['AddressID']=dfMerged01.apply(lambda x: dicLastChanceMaps[x['CICode']]  if (pd.isna(x['AddressID'])) & (x['CICode'] in dicLastChanceMaps) else x['AddressID'],axis=1)
    return dfMerged01
    
def BusinessUnitsExtracts():
    dfRapidMinerBusinessUnits = GetCombinedFIMsEvo()
    dfRapidMinerBusinessUnits.to_csv(DIR_RAPIDMINER + '\\BusUnits.csv',index=False)
    lstColsToDrop = ['Distributor CI Code','Franchise Country','JLR Number','Brand','Franchise CI Code','Franchise Trading Title','Franchise Type','Address Town','Address Postcode','Address ID','Franchise Status','F_CICode','ToRemove']
    dfGCPbusinessUnits = dfRapidMinerBusinessUnits.drop(labels=lstColsToDrop,axis=1)
    dfGCPbusinessUnits.to_csv(DIR_RAPIDMINER + '\\BusUnitsGCP.csv',index=False)


   
    
if __name__ == '__main__':
    print('Running Module NERA_BAU_EXTRACTS_BUS_UNITS_01 at ' + datetime.now().strftime('%H:%M:%S'))
    BusinessUnitsExtracts()
    print('Finished NERA_BAU_EXTRACTS_BUS_UNITS_01 at ' + datetime.now().strftime('%H:%M:%S'))

In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Wed Sep  7 09:20:10 2022

@author: MREID7
"""

import json
import pandas as pd
import sqlalchemy as sqa
from lxml import etree
from datetime import datetime
#from lxml.objectify import NoneElement
#import lxml
import NERA_BAU_STATIC_DATA_01 as STATIC_DATA

print(STATIC_DATA.Username())
DATE_FORMAT = '%Y-%m-%d %H:%M:%S'


USER_DOCS_PATH = 'C:\\Users\\' + STATIC_DATA.Username() + '\\OneDrive\\OneDrive - JAGUAR LAND ROVER\\Documents'
NERA_REPOS_PATH = 'C:\\Users\\' + STATIC_DATA.Username() + r'\OneDrive\JAGUAR LAND ROVER\Network Excellence Reporting & Analytics - NERA Data Repository'
FILE_FIMS_SPEC = NERA_REPOS_PATH + '\\Raw Extracts\\FIMS - Franchise Specialism Report\Franchise_Specialism_Report.xlsx'
FIL_STATIC = NERA_REPOS_PATH + '\\Reference Data\Business Units\\FIMsStaticMapping.json'
DIR_RAPIDMINER = NERA_REPOS_PATH + '\\Raw Extracts\\EvoReportingDB'

DIR_TEST_REPOSITORY ='C:\\temp\\'
#FILE_MKT_GRP = 'tblMarketGroup.csv'
#FILE_MKTS = 'tblMarket.csv'
#FILE_BUTMMG ='tblBusinessUnitToMarketOrMarketGroup.csv'
FILE_BUS_UNITS = 'BusUnits.csv'

DB_DESC = 'V2E_Reporting_UAT' #'V2E_Warehouse_UAT' #'V2E_Reporting_UAT' #'V2E_Reporting_PROD'
FILE_CREDENTIALS = 'local_configs.json'

def GetDBengine(sKeyDB,sCredsPath,sCredsFile):
    #Creates a database connection and SQLAlchemy engine from User Config file
    crdntlspath = sCredsPath + '\\' + sCredsFile
    print(crdntlspath)
    with open(crdntlspath,'r') as read_crdntls:
        crdntls = json.load(read_crdntls)
        dbentry = crdntls['logins'][sKeyDB]
    connStr = 'mysql+pymysql://' + dbentry['user'] + ':' + dbentry['pass'] + '@' + dbentry['ip'] + ':' + dbentry['port'] + '/' + dbentry['name']
    objEngine = sqa.create_engine(connStr)
    return objEngine

#Map Users to Market
def UsersToMarketMap():
    engine = GetDBengine(DB_DESC,USER_DOCS_PATH,FILE_CREDENTIALS)
    strDirData = STATIC_DATA.DIR_RAW_EVO_EXTRACTS()
    strSQL = STATIC_DATA.STR_SQL_USERS_TO_MKT_MKTGRP
    dfExtract = pd.read_sql(strSQL,engine)
    dfExtract.to_csv(strDirData + '\\' + 'UsersToMarketMap.csv',index=False)

#Market Groups

class BusinessUnit:
    def __init__(self,strUUID):
        self.Name = ''
        self.UUID = strUUID
        self.CICode = ''
        self.FranchiseType = ''

class Market:
    def __init__(self,strUUID,strName):
        self.Name = strName
        self.UUID = strUUID
        self.BusinessUnits = {}
            
    def AddNewBusinessUnit(self, strUUID, objBU):
        if strUUID not in self.BusinessUnits:
            #print('Adding BusUnit')
            self.BusinessUnits[strUUID] = objBU
        
    def GenerateXmlMarketElement(self):
        lmRoot = etree.Element('itm', level='m', uuid=self.UUID , nm=self.Name)
        if len(self.BusinessUnits)>0:
            #print('Market has Bus Units')
            lmChildren = etree.Element('itms')
            for sBU in self.BusinessUnits:
                oBU = self.BusinessUnits[sBU]
                lmBU = etree.Element('itm', level='b', uuid=sBU , nm=oBU.Name)
                lmChildren.append(lmBU)
            lmRoot.append(lmChildren)
        return lmRoot
    
class MarketGroup(Market):
    def __init__(self,strUUID,strName):
        Market.__init__(self,strUUID,strName)
        self.SubGroups = {}
        self.Markets = {}

        
    def AddNewSubGroup(self, strUUID, objMktGrp):
        if strUUID not in self.SubGroups:
            self.SubGroups[strUUID] = objMktGrp

    def AddNewMarket(self, strUUID, objMarket):
        if strUUID not in self.Markets:
            self.Markets[strUUID] = objMarket
            
    def GenerateXmlMarketGroupElement(self):
        lmRoot = etree.Element('itm', level='mg', uuid=self.UUID , nm=self.Name)
        if len(self.BusinessUnits)>0 or len(self.Users)>0 or len(self.SubGroups)>0 or len(self.Markets)>0:
            lmChildren = etree.Element('itms')
            for sSG in self.SubGroups:
                oSG = self.SubGroups[sSG]
                lmSG = oSG.GenerateXmlMarketGroupElement()
                lmChildren.append(lmSG)
            for sM in self.Markets:
                oM = self.Markets[sM]
                lmM = oM.GenerateXmlMarketElement()
                lmChildren.append(lmM)
            for sBU in self.BusinessUnits:
                oBU = self.BusinessUnits[sBU]
                lmBU = etree.Element('itm', level='b', uuid=sBU , nm=oBU.Name)
                lmChildren.append(lmBU) 
            lmRoot.append(lmChildren)
        return lmRoot
    
def GroupTree01(uuid, name,dfMktGrps,dfMkts,dfMktMap,dicMktBUs):
    #print('Name: ' + name)
    objMG = MarketGroup(uuid,name)
    if uuid in dicMktBUs:
        for sBU in dicMktBUs[uuid]:
            objMG.AddNewBusinessUnit(sBU,dicMktBUs[uuid][sBU])
    for tupMkt in dfMkts[dfMkts['MarketGroupUUID']==uuid].itertuples():
        oMkt = Market(tupMkt.MarketUUID,tupMkt.MarketName)  
        if tupMkt.MarketUUID in dicMktBUs:
            for sBU in dicMktBUs[tupMkt.MarketUUID]:
                oMkt.AddNewBusinessUnit(sBU,dicMktBUs[tupMkt.MarketUUID][sBU])
        objMG.AddNewMarket(tupMkt.MarketUUID,oMkt)
    for tupMG in dfMktGrps[dfMktGrps['ParentMarketGroupUUID']==uuid].itertuples():
        oChild = GroupTree01(tupMG.MarketGroupUUID,tupMG.MarketGroupName,dfMktGrps,dfMkts,dfMktMap,dicMktBUs)
        objMG.AddNewSubGroup(tupMG.MarketGroupUUID, oChild)
    return objMG

def BuildXmlDoc01(objBase, lm):
    lmChild = etree.Element('mg', uuid=objBase.UUID, nm=objBase.Name)
    for sBU in objBase.BusinessUnits:
        #print('In MktGrp {0}, {1} add BU: {2}'.format(objBase.UUID,objBase.Name,sBU))
        objBU = objBase.BusinessUnits[sBU] 
        #print(' - BU: {0}, {1}, {2}, {3}'.format(objBU.UUID,objBU.Name, objBU.CICode,objBU.FranchiseType))
        lmBU = etree.Element('bu', uuid=str(objBU.UUID), nm=str(objBU.Name), ci=str(objBU.CICode),ftyp=str(objBU.FranchiseType) )
        lmChild.append(lmBU)
    if len(objBase.Markets)>0:
        lmMkts = etree.Element('mkts')
        lmChild.append(lmMkts)
        for sMkt in objBase.Markets:
            objMkt = objBase.Markets[sMkt]
            lmMkt = etree.Element('mkt', uuid=objMkt.UUID, nm=objMkt.Name)
            for sBU in objMkt.BusinessUnits:
                #print('In Mkt add BU: {0}'.format(sBU))
                objBU = objMkt.BusinessUnits[sBU] 
                #print('..adding...uuid: {0} ... Name: {1}, CIcode {2}, {3}'.format(objBU.UUID,objBU.Name, objBU.CICode,objBU.FranchiseType))
                lmBU = etree.Element('bu', uuid=objBU.UUID, nm=objBU.Name, ci=objBU.CICode, ftyp=objBU.FranchiseType)
                lmMkt.append(lmBU)
            lmMkts.append(lmMkt)
        lmChild.append(lmMkts)
    lm.append(lmChild)
    if len(objBase.SubGroups)>0:
        lmMgs = etree.Element('mgs')
        lmChild.append(lmMgs)
        for sSubG in objBase.SubGroups:
            objSubG=objBase.SubGroups[sSubG]
            BuildXmlDoc01(objSubG,lmMgs)
    


def MarketGroups(lmUnit,iListLength): 
    nlMktGrps = lmUnit.xpath("ancestor::mg")
    #print(len(nlMktGrps))
    strTopLevelName = nlMktGrps[0].attrib['nm']
    strTopLevelUUID = nlMktGrps[0].attrib['uuid']
    if len(nlMktGrps) < (iListLength + 1):
        lstUnits = [strTopLevelName] * (iListLength - len(nlMktGrps) - 1)
        lstUUIDs = [strTopLevelUUID] * (iListLength - len(nlMktGrps) - 1)
    else:
        lstUnits = []
        lstUUIDs = []
    for lmMG in nlMktGrps:
        #print(lmMG.attrib['nm'])
        lstUnits.append(lmMG.attrib['nm'])
        lstUUIDs.append(lmMG.attrib['uuid'])
    lstUnits.append(lmUnit.attrib['nm'])
    lstUUIDs.append(lmUnit.attrib['uuid'])
    #print(lstUnits)
    return lstUnits + lstUUIDs



def MarketGroupsXml():
    engine = GetDBengine(DB_DESC,USER_DOCS_PATH,FILE_CREDENTIALS)
    dfMtkGrp = pd.read_sql("SELECT MarketGroupUUID,MarketGroupName,ParentMarketGroupUUID,Level,MarketGroupStatusUUID,IsBusinessRegion FROM MarketGroup WHERE MarketGroupName NOT IN ('Historical Field Teams','ETL Test','Test Market Group','Historical Global Trainers','Historical JLRE')",engine)
    dfMarkets = pd.read_sql('SELECT MarketUUID,BusinessRegionUUID,MarketGroupUUID,MarketName,CONV(IsActive,2,10) as IsActive,MarketStatusUUID, CONV(IsTestMarket,2,10) as IsTestMarket FROM Market',engine)
    dfBUTMMG = pd.read_sql('SELECT BusinessUnitToMarketOrMarketGroupUUID,BusinessUnitUUID,MarketOrMarketGroupUUID FROM BusinessUnitToMarketOrMarketGroup',engine)
    dfBusUnits = pd.read_csv(DIR_RAPIDMINER + '\\' + FILE_BUS_UNITS)
    dfBusUnits.fillna('',inplace=True)
    dicMktBUs = {}
    for mktUUID in dfBUTMMG['MarketOrMarketGroupUUID'].unique():
        dicBUs = {}
        for tupBU in dfBUTMMG[dfBUTMMG['MarketOrMarketGroupUUID']==mktUUID].itertuples():
            oBU = BusinessUnit(tupBU.BusinessUnitUUID)
            dfTmpBU = dfBusUnits[dfBusUnits['BusinessUnitUUID']==tupBU.BusinessUnitUUID].reset_index()
            if len(dfTmpBU.index)>0:
                oBU.Name = dfTmpBU.loc[0,'BusinessUnit']
                oBU.CICode = dfTmpBU.loc[0,'CICode']
                oBU.FranchiseType = dfTmpBU.loc[0,'BusUnitSubType']
            dicBUs[tupBU.BusinessUnitUUID] = oBU
        dicMktBUs[mktUUID]=dicBUs
    dfGlobal = dfMtkGrp[dfMtkGrp['ParentMarketGroupUUID'].isnull()].iloc[0]
    objGlobal = GroupTree01(dfGlobal['MarketGroupUUID'],dfGlobal['MarketGroupName'],dfMtkGrp,dfMarkets,dfBUTMMG,dicMktBUs)
    lmRoot = etree.Element('root')
    lmTopLevel = etree.Element('mgs')
    lmRoot.append(lmTopLevel)
    BuildXmlDoc01(objGlobal,lmTopLevel)
    return etree.ElementTree(lmRoot)

def MarketGroupsXmlToFlat():
    INT_MAX_LEVEL = 5
    tree = MarketGroupsXml()
    nlMkts = tree.findall(".//mkt")
    lstData = []
    for lmMkt in nlMkts:
        lstFull = MarketGroups(lmMkt,INT_MAX_LEVEL)
        lstData.append(lstFull)
    dfMKtGrps = pd.DataFrame(lstData)
    dfMKtGrps.to_csv(DIR_RAPIDMINER + '\\MarketGroups.csv',index=False)
    #dfMKtGrps.to_csv('C:\\temp\\MarketGroups.csv',index=False)
    
if __name__ == '__main__':
    print('Running Module NERA_BAU_EXTRACTS_REGIONS_01 at ' + datetime.now().strftime('%H:%M:%S'))
    MarketGroupsXmlToFlat()
    print('Completed Market Groups, now UsersToMarketMap at ' + datetime.now().strftime('%H:%M:%S'))
    UsersToMarketMap()
    print('Finished NERA_BAU_EXTRACTS_REGIONS_01 at ' + datetime.now().strftime('%H:%M:%S'))


In [13]:
wordlist = "myname is a Idris Adebisi.I am destined to be successful in life and I am working towards that..may God continue to guide and direct me and keep my family alive and make them enjoy thier time on earth..may God also forgive all my sins and make me achaive everything i am for including my wife and my siblings and parents"

In [10]:
import re

In [134]:
email = """Mr. Michael
            Mr Smith
            Ms Kritika
            Mrs. T
            Ms. Leena
        """

In [158]:
a = re.compile(r'\w+\.?\W\w+')
m = re.compile(r'Mr.?\W\w+')
f = re.compile(r'Mr?s.?\W\w+')

In [138]:
a = re.compile(r'\w+\.?\w+@\w+.\w+')

In [144]:
list(a.finditer(email))

[<re.Match object; span=(0, 11), match='Mr. Michael'>,
 <re.Match object; span=(24, 32), match='Mr Smith'>,
 <re.Match object; span=(45, 55), match='Ms Kritika'>,
 <re.Match object; span=(68, 74), match='Mrs. T'>,
 <re.Match object; span=(87, 96), match='Ms. Leena'>]

In [156]:
from graphviz import Digraph
def trace(root):
    nodes, edges = set(), set()
    def build(v):
        if v not in nodes:
            nodes.add(v)
            for child in v._downstream:
                edges.add((child, v))
                build(child)
    build(root)
    return nodes, edges

def draw_dot(root, format='svg', rankdir='LR'):
    """
    format: png | svg | ...
    rankdir: TB (top to bottom graph) | LR (left to right)
    """
    assert rankdir in ['LR', 'TB']
    nodes, edges = trace(root)
    dot = Digraph(format=format, graph_attr={'rankdir': rankdir}) #, node_attr={'rankdir': 'TB'})
    
    for n in nodes:
        dot.node(name=str(id(n)), label = "{ data %.4f | grad %.4f }" % (n.data, n.grad), shape='record')
        if n._op:
            dot.node(name=str(id(n)) + n._op, label=n._op)
            dot.edge(str(id(n)) + n._op, str(id(n)))
    for n1, n2 in edges:
        dot.edge(str(id(n1)), str(id(n2)) + n2._op)
    return dot

In [171]:
import numpy as np

class Tensor:
    __allnodes = []
    def __init__(self, data, _downstream=(), _op='', dtype=None):
        assert isinstance(data , (int,float)), "data must be of type int or float"
        self.data = float(data) if not dtype else dtype(data)
        self._downstream = _downstream
        self._backward = lambda: None
        self._op = _op
        self.grad = 0.0

    def __add__(self, other):
       other = other if isinstance(other, Tensor) else Tensor(other)
       out =  Tensor(data =(self.data + other.data), _downstream=(self,other), _op= '+')
       def _backward():
          self.grad += 1.0 * out.grad
          other.grad += 1.0 * out.grad
       out._backward = _backward
       return out  

    def __mul__(self, other):
       other = other if isinstance(other, Tensor) else Tensor(other)
       out =  Tensor(data =(self.data * other.data), _downstream=(self,other), _op= '*') 
       def _backward():
         self.grad += other.data * out.grad
         other.grad += self.data * out.grad
       out._backward = _backward
       return out 
    
    def __pow__(self, other):
       other = other if isinstance(other, Tensor) else Tensor(other)
       out = Tensor(data = (self.data**other.data), _downstream=(self,), _op='pow')
       def _backward():
         self.grad += (other.data) * (self.data ** (other.data -1)) * out.grad
         other.grad += (self.data) * (other.data ** (self.data -1)) * out.grad
       out._backward = _backward
       return out
    
    def relu(self):
        out = Tensor(data=(0 if self.data < 0 else self.data), _downstream=(self,), _op='relu')
        def _backward():
            self.grad += out.data * out.grad
        out._backward = _backward
        return out
    
    @classmethod
    def tanh(self):
        t = ((2*self).exp() -1) / ((2*self).exp()+ 1)
        out = Tensor(data=(t.data), _downstream=(self, ),  _op='tanh')
        def _backward():
            self.grad += (1 - t.data**2) * out.grad
        out._backward = _backward
        return out
       
    def sigmoid(self):
        x = 1 / (1 + -self.exp())
        out = Tensor(data=(x.data), _downstream=(self, ),  _op='sigmoid')
        def _backward():
            self.grad = None
        out._backward = _backward
        return out
    
    def exp(self):
        x = self.data
        out = Tensor(data=(np.exp(x)), _downstream=(self, ), _op='exp')
        def _backward():
            self.grad += out.data * out.grad
        out._backward = _backward        
        return out   
       
    def backward(self):
       nodes = self.__allnodes()
       self.grad = 1
       for node in nodes:
         node._backward()
   
    def zero_grad(self):
       nodes = self.__allnodes()
       for node in nodes:
         node.grad = 0
    
    def __allnodes(self):
       nodes = []
       def get_downstream(object, visited = set()):
         if object not in visited:
            visited.add(object)
            nodes.append(object)
            for children in object._downstream:
               get_downstream(children)
       get_downstream(self)
       return nodes 
    
    def __neg__(self): 
       return self * -1

    def __radd__(self, other):
       return self + other
    
    def __truediv__(self, other):
       return self * (other**-1)
    
    def __rmul__(self,other):
       return self * other
    
    def __sub__(self, other):
      return self + (-other)
    
    def __rtruediv__(self, other):
       return self * (other**-1)
    
    def __repr__(self):
      return f"Tensor(data= {self.data}, grad={self.grad})"

In [647]:
class Neuron:
    def __init__(self, inx):
        """"
        inx: number of input to the neuron
        """
        self.weight = [Tensor(np.random.normal()) for _ in range(inx)]
        self.bias = Tensor(np.random.normal())
    
    def __call__(self, x):
        out = sum((x*w for x,w in zip(x,self.weight))) + self.bias
        # out = out.relu()
        return out
    
    def parameters(self):
        return self.weight + [self.bias]

class LinearLayer:
    def __init__(self, inx, out):
        """"
        inx: number of neuron input in the input layer
        out: number of neurons in the layer
        """
        self.neurons = [Neuron(inx) for _ in range(out)]       
    def __call__(self, x):
        out = [n(x) for n in self.neurons]
        return out[0] if len(out) < 2 else out
    
    def parameters(self):
        return [p for neuron in self.neurons for p in neuron.parameters() ]
    
class Sequential:
    def __init__(self, layers):
        self.layers = layers

    def __call__(self, x):
        for layer in self.layers:
            x = layer(x)
        return x
    
    def parameters(self):
        return [p for layer in self.layers for p in layer.parameters()]
    

from abc import abstractmethod, ABCMeta
class Model(metaclass=ABCMeta):
    def __init__(self, *args, **kwargs):
        super().__init__()

    @abstractmethod
    def forward(self,*args, **kwargs):
        raise NotImplementedError
    
    def parameters(self):
        layers = self._layers()
        parameters = [p for layer in layers for p in layer.parameters()]
        return parameters

    def _layers(self):
        layers = []
        for _, layer in  self.__dict__.items():
            if isinstance(layer, (LinearLayer,)):
                layers.append(layer)
        return layers

    def summary(self):
        layers = self._layers()


class MLP(Model):
    def __init__(self):
        super(MLP,self).__init__()
        self.layer = LinearLayer(3,4)

    def forward(self, x):
        x = self.layer(x)
        return x

In [ ]:
class Activation:
    @classmethod
    def tanh(cls,x):
        out = ((2*x).exp() -1) / ((2*x).exp()+ 1)
        return out
    
    @classmethod    
    def sigmoid(cls, x):
        out = 1 / (1 + -x.exp())
        return out  

class Loss:
    @classmethod
    def bianaryentropy(cls, ytrue, ypred):
        loss = (-ytrue * np.log(ypred)) + ((1-ytrue)*np.log(1-ypred))
        return loss

In [651]:
b = Tensor(5)

In [652]:
w = Activation.tanh(b)

In [654]:
w

Tensor(data= 0.9999092042625952, grad=0.0)

In [634]:
a = w._layers()

In [606]:
for w , v in w.__dict__.items():
    print(w)
    print(v)

layer


In [ ]:
def tanh(self):
    t = ((2*self).exp() -1) / ((2*self).exp()+ 1)
    out = Tensor(data=(t.data), _downstream=(self, ),  _op='tanh')
    def _backward():
        self.grad += (1 - t.data**2) * out.grad
    out._backward = _backward
    return out
    
def sigmoid(self):
    x = 1 / (1 + -self.exp())
    out = Tensor(data=(x.data), _downstream=(self, ),  _op='sigmoid')
    def _backward():
        self.grad = None
    out._backward = _backward
    return out

In [522]:
model = Sequential([LinearLayer(3,4), LinearLayer(4,4), LinearLayer(4,1)])

In [575]:
for i in range(50):
    ypred = [model(x) for x in xs]
    loss = sum((yhat-y)**2 for yhat,y in zip(ypred,ys)) / len(ypred)
    if i in range(0,500,25):
        print(f'Iteration:{i} ---> {loss.data:}')
    lr = 0.01
    loss.zero_grad()
    loss.backward()
    for p in model.parameters():
        p.data += -(lr * p.grad)

In [155]:
import numpy as np

class Tensor:
    """
    A class representing a tensor with a single numerical value.

    Parameters:
    -----------
    data : int or float
        The numerical value of the tensor.
    _downstream : tuple of Tensors, optional
        Tensors that depend on this tensor. Defaults to an empty tuple.
    _op : str, optional
        The operation performed on this tensor. Defaults to an empty string.
    dtype : type, optional
        The data type of the tensor. Defaults to None.

    Attributes:
    -----------
    data : float
        The numerical value of the tensor.
    grad : float
        The gradient of the tensor with respect to the loss.
    _downstream : tuple of Tensors
        Tensors that depend on this tensor.
    _op : str
        The operation performed on this tensor.

    Methods:
    --------
    __init__(self, data, _downstream=(), _op='', dtype=None):
        Initializes a new instance of the Tensor class.
    __add__(self, other):
        Computes the sum of two tensors.
    __mul__(self, other):
        Computes the element-wise multiplication of two tensors.
    __pow__(self, other):
        Computes the element-wise power of a tensor.
    relu(self):
        Computes the Rectified Linear Unit (ReLU) activation function of a tensor.
    tanh(self):
        Computes the hyperbolic tangent (tanh) activation function of a tensor.
    sigmoid(self):
        Computes the sigmoid activation function of a tensor.
    exp(self):
        Computes the exponential function of a tensor.
    backward(self):
        Computes the gradient of all the tensors in the computational graph with respect to some scalar loss function.
    zero_grad(self):
        Sets the gradient of all the tensors in the computational graph to zero.
    __allnodes(self):
        Returns a list of all the tensors in the computational graph.
    __neg__(self):
        Computes the negative value of a tensor.
    __radd__(self, other):
        Computes the sum of a tensor and a scalar value.
    __truediv__(self, other):
        Computes the element-wise division of two tensors.
    __rmul__(self,other):
        Computes the element-wise multiplication of a tensor and a scalar value.
    __sub__(self, other):
        Computes the difference between two tensors.
    __rtruediv__(self, other):
        Computes the element-wise division of a scalar value and a tensor.
    __repr__(self):
        Returns a string representation of the tensor.
    """
    
    def __init__(self, data, _downstream=(), _op='', dtype=None):
        """
        Initialize a Tensor object.
        Parameters:
            data (int or float): The numerical value of the tensor.
            _downstream (tuple, optional): A tuple of downstream tensors.
            _op (str, optional): The operation that produced this tensor.
            dtype (type, optional): The desired data type of the tensor.
        Raises:
            AssertionError: If `data` is not of type `int` or `float`.
        """
        assert isinstance(data , (int,float)), "data must be of type int or float"
        self.data = float(data) if not dtype else dtype(data)
        self._downstream = _downstream
        self._op = _op
        self.grad = 0.0

    def __repr__(self):
        """
        Return a string representation of the Tensor object.
        """
        return f"Tensor({self.data})"

    def __add__(self, other):
        """
        Return the sum of this tensor and another tensor or numerical value.
        Parameters:
            other (Tensor or int or float): The other tensor or numerical value to add.
        Returns:
            Tensor: A new tensor representing the sum of this tensor and `other`.
        """
        other = other if isinstance(other, Tensor) else Tensor(other)
        out =  Tensor(data =(self.data + other.data), _downstream=(self,other), _op= '+')   
        return out  

    def __mul__(self, other):
        """
        Return the product of this tensor and another tensor or numerical value.
        Parameters:
            other (Tensor or int or float): The other tensor or numerical value to multiply.

        Returns:
            Tensor: A new tensor representing the product of this tensor and `other`.
        """
        other = other if isinstance(other, Tensor) else Tensor(other)
        out =  Tensor(data =(self.data * other.data), _downstream=(self,other), _op= '*') 
        return out 
    
    def __pow__(self, other):
        """
        Return this tensor raised to the power of another numerical value.

        Parameters:
            other (int or float): The numerical value to raise this tensor to the power of.

        Returns:
            Tensor: A new tensor representing this tensor raised to the power of `other`.
        """
        out = Tensor(data = (self.data**other), _downstream=(self,), _op='pow')
        return out
    
    def relu(self):
        """
        Return the Rectified Linear Unit (ReLU) of this tensor.

        Returns:
            Tensor: A new tensor representing the ReLU of this tensor.
        """
        x = self.data
        out = Tensor(0, (self,), 'relu') if x < 0 else x
        return out
    
    def tanh(self):
        """
        Return the hyperbolic tangent (tanh) of this tensor.

        Returns:
            Tensor: A new tensor representing the hyperbolic tangent of this tensor.
        """
        t = ((2*self).exp() -1) / ((2*self).exp()+ 1)
        out = Tensor(t.data, (self, ), 'tanh')
        return out
       
    def sigmoid(self):
        """
        Return the sigmoid function of this tensor.

        Returns:
            Tensor: A new tensor representing the sigmoid function of this tensor.
        """
        x = 1 / (1 + -self.exp())
        out = Tensor


[-0.6931471805599453,
 0.6931471805599453,
 0.6931471805599453,
 0.6931471805599453,
 -0.6931471805599453,
 -0.6931471805599453,
 -0.6931471805599453,
 -0.6931471805599453]

In [156]:
preds

[0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5]